# Video game clustering

## Preprocessing the data

* Handle missing data
* Use KNN for imputation
* Save processed data

In [1]:
# Core libraries
import pandas as pd
import numpy as np

# Imputation of missing values
from fancyimpute import KNN

# Models
from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

Data columns:

| Column           | Description                           |
|------------------|---------------------------------------|
| name             | Name of the game                      |
| release_date     | Date of when the game was released    |
| developer        | Game developer                        |
| achievements     | How many achievements are in the game |
| original_price   | Price of the game                     |
| percent_positive | Percentage of positive reviews        |
| num_reviews      | Number of reviews                     |
| multiplayer      | Is the game multiplayer or not        |
| mature           | Is the game suitable for kids or not  |

In [2]:
data = pd.read_csv("video_games.csv")

## EDA

In [3]:
data.head()

,name,release_date,developer,achievements,original_price,percent_positive,num_reviews,multiplayer,mature
0,DOOM,"May 12, 2016",id Software,54.0,$19.99,92%,42550.0,True,False
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",PUBG Corporation,37.0,$29.99,49%,836608.0,True,True
2,BATTLETECH,"Apr 24, 2018",Harebrained Schemes,128.0,$39.99,71%,7030.0,True,False
3,DayZ,"Dec 13, 2018",Bohemia Interactive,NaN,$44.99,61%,167115.0,True,False
4,EVE Online,"May 6, 2003",CCP,NaN,Free,74%,11481.0,True,False


In [4]:
data.dtypes

name                 object
release_date         object
developer            object
achievements        float64
original_price       object
percent_positive     object
num_reviews         float64
multiplayer          object
mature                 bool
dtype: object

### Data columns

* name. Text string. Game title.
* release_date (Drop this column). Text string. The year could potentially be used as a clustering feature but in this first version, it will be removed. 
* developer. Text string. 
* achievements (Drop this column). Floating point number. No relation with determining the game genre.
* original_price (Drop this column). Text string. Requires careful processing to extract the price as an integer. Dropped from analysis for now but would like to include in later versions.
* percent_positive. Text string. 
* num_reviews. Floating point number.
* multiplayer. Text string. 
* mature. Boolean.

### Version 1 model

* Retain: name, developer, percent_positive, num_reviews, multiplayer, mature
* Remove: release_date, achievements, original_price

In [5]:
for column in data.columns:
    print(column, data[column].unique()[:10])
    print("")


name ['DOOM' "PLAYERUNKNOWN'S BATTLEGROUNDS" 'BATTLETECH' 'DayZ' 'EVE Online'
 'Grand Theft Auto V: Premium Online Edition' 'Devil May Cry 5'
 'Human: Fall Flat' 'They Are Billions' 'Warhammer: Chaosbane']

release_date ['May 12, 2016' 'Dec 21, 2017' 'Apr 24, 2018' 'Dec 13, 2018' 'May 6, 2003'
 nan 'Mar 7, 2019' 'Jul 22, 2016' 'Dec 12, 2017' 'May 31, 2019']

developer ['id Software' 'PUBG Corporation' 'Harebrained Schemes'
 'Bohemia Interactive' 'CCP' 'Rockstar North' 'CAPCOM Co., Ltd.'
 'No Brakes Games' 'Numantian Games' 'Eko Software']

achievements [ 54.  37. 128.  nan  51.  55.  34.  43.  72.  41.]

original_price ['$19.99' '$29.99' '$39.99' '$44.99' 'Free' nan '$59.99' '$14.99' '$49.99'
 'Free to Play']

percent_positive ['92%' '49%' '71%' '61%' '74%' nan '91%' '85%' '44%' '83%']

num_reviews [ 42550. 836608.   7030. 167115.  11481.     nan   9645.  23763.  12127.
    904.]

multiplayer [True False nan]

mature [False  True]



## Data preprocessing

### Drop columns

* Remove: release_date, achievements, original_price

In [6]:
data.drop(columns=['release_date', 'achievements', 'original_price'], inplace=True)

In [7]:
data.head()

,name,developer,percent_positive,num_reviews,multiplayer,mature
0,DOOM,id Software,92%,42550.0,True,False
1,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,49%,836608.0,True,True
2,BATTLETECH,Harebrained Schemes,71%,7030.0,True,False
3,DayZ,Bohemia Interactive,61%,167115.0,True,False
4,EVE Online,CCP,74%,11481.0,True,False


percent_positive

* Text string, format is an integer followed by the % sign
* Contains NaNs
* Strategy is to remove the % symbol and convert to integer
* Example "92%" is changed to 92.0

In [8]:
data['percent_positive'].unique()

array(['92%', '49%', '71%', '61%', '74%', nan, '91%', '85%', '44%', '83%',
       '84%', '78%', '51%', '75%', '94%', '77%', '88%', '97%', '70%',
       '90%', '63%', '81%', '60%', '96%', '87%', '68%', '86%', '100%',
       '95%', '80%', '93%', '82%', '52%', '72%', '79%', '98%', '64%',
       '69%', '89%', '54%', '76%', '73%', '55%', '65%', '59%', '45%',
       '57%', '67%', '62%', '46%', '66%', '27%', '56%', '37%', '53%',
       '99%', '42%', '50%', '48%', '58%', '41%', '43%', '40%', '34%',
       '47%', '38%', '25%', '15%', '33%', '39%', '21%', '30%', '23%',
       '24%', '35%', '28%', '36%', '31%', '29%', '20%', '9%', '0%', '10%',
       '17%', '8%', '26%', '32%', '18%', '13%', '22%', '19%', '4%', '14%',
       '5%', '6%', '7%', '16%', '12%', '3%', '2%', '11%'], dtype=object)

In [9]:
# Remove the % sign
data['percent_positive'] = data['percent_positive'].str.replace("%", "")

# Convert from text string to float. Filter on non-NaNs otherwise conversion won't work
data.loc[data['percent_positive'].notnull(), 'percent_positive'] = data.loc[data['percent_positive'].notnull(), 'percent_positive'].apply(float)

In [10]:
data['percent_positive']

0        92.0
1        49.0
2        71.0
3        61.0
4        74.0
         ... 
40828     NaN
40829     NaN
40830     NaN
40831     NaN
40832     NaN
Name: percent_positive, Length: 40833, dtype: object

### num_reviews

* Floating point values
* Contains NaNs

In [11]:
data['num_reviews']

0         42550.0
1        836608.0
2          7030.0
3        167115.0
4         11481.0
           ...   
40828         NaN
40829         NaN
40830         NaN
40831         NaN
40832         NaN
Name: num_reviews, Length: 40833, dtype: float64

In [12]:
data['num_reviews'].isnull().sum()

23470

### multiplayer

* Boolean. 29233 False. 11080 True
* Contains 520 NaNs

In [13]:
data['multiplayer'].value_counts()

False    29233
True     11080
Name: multiplayer, dtype: int64

In [14]:
data['multiplayer'].isnull().sum()

520

### mature

* Boolean. 37936 False. 2897 True
* No NaNs

In [15]:
data['mature'].value_counts()

False    37936
True      2897
Name: mature, dtype: int64

### Handle missing values

* "Name" column has 16 empty values. We cannot reconstruct the missing names so the rows are ignored
* Drop rows that have no values across four key features: developer, multiplayer, percent_positive, num_reviews
* Convert categorical labels to ordinal
* Use K-nearest neighbours to impute missing values

In [16]:
# "Name" has 16 rows that are empty. We cannot reconstruct the missing names so the rows are ignored  
data.isnull().sum()

name                   16
developer             343
percent_positive    23470
num_reviews         23470
multiplayer           520
mature                  0
dtype: int64

In [17]:
# Ignore the empty names
nans = data['name'].isnull()
data = data[~nans]

In [18]:
data.isnull().sum()

name                    0
developer             327
percent_positive    23454
num_reviews         23454
multiplayer           508
mature                  0
dtype: int64

In [19]:
# Check where there are NaNs across 4 feature columns: developer, multiplayer, percent_positive, num_reviews
nan_developer = data['developer'].isnull()
nan_multiplayer = data['multiplayer'].isnull()
nan_percentpositive = data['percent_positive'].isnull()
nan_numreviews = data['num_reviews'].isnull()

mask =  nan_developer & nan_multiplayer & nan_percentpositive & nan_numreviews
data = data[~mask]
data.reset_index(drop=True, inplace=True)

In [20]:
data.head()

,name,developer,percent_positive,num_reviews,multiplayer,mature
0,DOOM,id Software,92.0,42550.0,True,False
1,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,49.0,836608.0,True,True
2,BATTLETECH,Harebrained Schemes,71.0,7030.0,True,False
3,DayZ,Bohemia Interactive,61.0,167115.0,True,False
4,EVE Online,CCP,74.0,11481.0,True,False


In [21]:
# Encode categorical values
categorical_columns = ['name', 'developer', 'multiplayer', 'mature']
enc = OrdinalEncoder()
data[categorical_columns] = enc.fit_transform(data[categorical_columns])

In [22]:
data.head(10)

,name,developer,percent_positive,num_reviews,multiplayer,mature
0,7498.0,16796.0,92.0,42550.0,1.0,0.0
1,23455.0,10728.0,49.0,836608.0,1.0,1.0
2,2749.0,6329.0,71.0,7030.0,1.0,0.0
3,7963.0,2031.0,61.0,167115.0,1.0,0.0
4,9902.0,2343.0,74.0,11481.0,1.0,0.0
5,14549.0,12240.0,NaN,NaN,1.0,0.0
6,8606.0,2332.0,92.0,9645.0,1.0,1.0
7,16290.0,10141.0,91.0,23763.0,1.0,0.0
8,35173.0,10282.0,85.0,12127.0,0.0,0.0
9,38234.0,4440.0,44.0,904.0,1.0,1.0


In [23]:
# Impute missing values using K-nearest neighbour
#   Select K nearest or similar data points using all non-missing features
#   Take average of the selected data points to impute the missing feature
imputer = KNN(verbose=False)
data.iloc[:, :] = np.round(imputer.fit_transform(data))

In [24]:
# Reverse the ordinal encoding to get the original labels
data[categorical_columns] = enc.inverse_transform(data[categorical_columns])

In [25]:
data.head()

,name,developer,percent_positive,num_reviews,multiplayer,mature
0,DOOM,id Software,92.0,42550.0,True,False
1,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,49.0,836608.0,True,True
2,BATTLETECH,Harebrained Schemes,71.0,7030.0,True,False
3,DayZ,Bohemia Interactive,61.0,167115.0,True,False
4,EVE Online,CCP,74.0,11481.0,True,False


In [26]:
# Confirm no missing data
data.isnull().sum()

name                0
developer           0
percent_positive    0
num_reviews         0
multiplayer         0
mature              0
dtype: int64

## Save data

In [27]:
data.to_csv("video_games_processed.csv", index=False)